#### IMPORTAÇÕES

In [1]:
import os
import pandas as pd
import numpy as np
from IPython.display import display, Markdown
import datetime as dt
from datetime import datetime
from sklearn.ensemble import GradientBoostingRegressor

#### FUNÇÕES TRIVIAIS

In [2]:
def input_periodo_de_contagio(df):
    lista_regiao = list(df["Country/Region"].unique())
    lista_df = []
    for i in lista_regiao:
        mask = df['Country/Region'] == i
        df_temp = df.loc[mask]
        dias_de_contagio = np.arange(1,df_temp.shape[0]+1)
        df_temp.insert(2, "Time", dias_de_contagio )
        lista_df.append(df_temp)
    df_final = pd.concat(lista_df) 
    return df_final

def drop_country(df,lista_atributos):
    for i in lista_atributos:
        df_temp = df[df[i].isnull()]
        lista_regiao_para_dropar = list(df_temp["Country/Region"].unique())
        df_filtrado = df[df['Country/Region'].isin(lista_regiao_para_dropar) == False]
    return df_filtrado

#### LEITURA DO BANCO DE DADOS REAL

In [3]:
nome_arquivo_analise = "complete_mundial_estadual_model_20200807.csv"
arquivo = os.path.abspath("real_datasets/"+nome_arquivo_analise)
dados = pd.read_csv(arquivo,  sep="," , decimal=".")

In [4]:
display(Markdown('<h4>' + "NOMES DOS ATRIBUTOS E O SEUS PRIMEIROS VALORES"+ '</h4>'))

dados['Date'] = dados['Date'].apply(lambda x: dt.datetime.strptime(x,'%m/%d/%Y'))
dados

<h4>NOMES DOS ATRIBUTOS E O SEUS PRIMEIROS VALORES</h4>

,C1,C2,C3,C4,C5,C6,C7,C8,Confirmed,Country/Region,Date,Deaths,estagio,peak,postinflex,preinflex
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,Afghanistan,2020-02-24,0.0,5.0,133.0,154.0,112.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Afghanistan,2020-02-25,0.0,5.0,133.0,154.0,112.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Afghanistan,2020-02-26,0.0,5.0,133.0,154.0,112.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Afghanistan,2020-02-27,0.0,5.0,133.0,154.0,112.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Afghanistan,2020-02-28,0.0,5.0,133.0,154.0,112.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26177,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,311.0,SE,2020-08-02,17.0,4.0,139.0,192.0,86.0
26178,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,444.0,SE,2020-08-03,24.0,4.0,139.0,192.0,86.0
26179,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,590.0,SE,2020-08-04,20.0,4.0,139.0,192.0,86.0
26180,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,917.0,SE,2020-08-05,16.0,4.0,139.0,192.0,86.0


#### VERIFICANDO A QUANTIDADE DE DADOS FALTANTES

In [5]:
dados.isnull().sum()

C1                0
C2                0
C3                0
C4                0
C5                0
C6                0
C7                0
C8                0
Confirmed         0
Country/Region    0
Date              0
Deaths            0
estagio           0
peak              0
postinflex        0
preinflex         0
dtype: int64

#### ORGANIZANDO OS DADOS PELA DATA E REGIÃO

In [6]:
# ORGANIZANDO OS DADOS PELA DATA E REGIÃO
dados = dados.sort_values(by=["Country/Region","Date"])
# RETIRADA DOS PERÍODOS SEM INFORMAÇÃO DE CONTÁGIO
dados = dados[dados['Confirmed'].notna()].reset_index(drop=True)
# INPUTA O TEMPO DE CONTÁGIO NA COLUNA "TIME"
dados = input_periodo_de_contagio(dados)

#### REMOÇÃO DE PAÍSES COM POUCO REGISTRO DE ÓBITOS

In [7]:
filtro_regiao = dados[['Country/Region','Confirmed',"Deaths"]].groupby(['Country/Region']).sum().sort_values(by=["Deaths"],ascending=False)
intervalo = (filtro_regiao['Deaths'] >= 50 )
excluidos = filtro_regiao.loc[intervalo]
excluidos = excluidos.reset_index()
lista_excluidos = list(excluidos["Country/Region"])
intervalo = dados['Country/Region'].isin(lista_excluidos)
dados_confiaveis = dados.loc[intervalo]
dados_confiaveis = dados_confiaveis.reset_index(drop = True)

#### REMOÇÃO DE PAÍSES COM  DADOS NÃO CONFIÁVEIS

In [8]:
lista_baixa_confiabilidade = ["Cameroon","Kazakhstan","Kyrgyzstan","Zambia"]
intervalo = dados_confiaveis['Country/Region'].isin(lista_baixa_confiabilidade) == False
dados_confiaveis = dados_confiaveis.loc[intervalo]
dados_confiaveis

,C1,C2,Time,C3,C4,C5,C6,C7,C8,Confirmed,Country/Region,Date,Deaths,estagio,peak,postinflex,preinflex
0,0.0,0.0,1,0.0,1.0,0.0,0.0,0.0,0.0,3.0,AC,2020-03-19,0.0,5.0,93.0,136.0,50.0
1,0.0,1.0,2,1.0,1.0,1.0,0.0,1.0,1.0,4.0,AC,2020-03-20,0.0,5.0,93.0,136.0,50.0
2,0.0,1.0,3,1.0,1.0,1.0,0.0,1.0,1.0,2.0,AC,2020-03-21,0.0,5.0,93.0,136.0,50.0
3,0.0,1.0,4,1.0,1.0,1.0,0.0,1.0,1.0,2.0,AC,2020-03-22,0.0,5.0,93.0,136.0,50.0
4,0.0,1.0,5,1.0,1.0,1.0,0.0,1.0,1.0,0.0,AC,2020-03-23,0.0,5.0,93.0,136.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18707,1.0,1.0,108,0.0,0.0,0.0,0.0,0.0,1.0,7.0,Yemen,2020-07-26,5.0,1.0,149.0,173.0,124.0
18708,1.0,1.0,109,0.0,0.0,0.0,0.0,0.0,1.0,10.0,Yemen,2020-07-27,4.0,1.0,149.0,173.0,124.0
18709,1.0,1.0,110,0.0,0.0,0.0,0.0,0.0,1.0,12.0,Yemen,2020-07-28,1.0,1.0,149.0,173.0,124.0
18710,1.0,1.0,111,0.0,0.0,0.0,0.0,0.0,1.0,8.0,Yemen,2020-07-29,1.0,1.0,149.0,173.0,124.0


#### EXCLUINDO OS PAÍSES DOS QUAIS NÃO POSSUEM A CONTRAMEDIA C1

In [9]:
dados_confiaveis_filtro1 = drop_country(dados_confiaveis,["C1","C2","C3","C4","C5","C6","C7","C8"])

#### REGIÕES QUE APLICARAM AS CONTRAMEDIDAS  (UTILIZADOS PARA O TREINAMENTO)

In [10]:
lista_de_regioes = list(dados_confiaveis_filtro1["Country/Region"].unique())
lista_de_regioes

['AC',
 'AL',
 'Afghanistan',
 'Albania',
 'Algeria',
 'Andorra',
 'Argentina',
 'Australia',
 'Austria',
 'Azerbaijan',
 'BA',
 'Bahrain',
 'Bangladesh',
 'Belarus',
 'Belgium',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Brazil',
 'Bulgaria',
 'Burkina Faso',
 'Canada',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Congo (Brazzaville)',
 'Congo (Kinshasa)',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Denmark',
 'Djibouti',
 'Dominican Republic',
 'ES',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Estonia',
 'Ethiopia',
 'Finland',
 'France',
 'Gabon',
 'Germany',
 'Ghana',
 'Greece',
 'Guatemala',
 'Haiti',
 'Honduras',
 'Hungary',
 'India',
 'Indonesia',
 'Iran',
 'Iraq',
 'Ireland',
 'Israel',
 'Italy',
 'Japan',
 'Kenya',
 'Kosovo',
 'Kuwait',
 'Liberia',
 'Libya',
 'Lithuania',
 'Luxembourg',
 'Madagascar',
 'Malawi',
 'Malaysia',
 'Mali',
 'Mauritania',
 'Mexico',
 'Moldova',
 'Morocco',
 'Netherlands',
 'Nicaragua',
 'Niger',
 'Nigeria',
 'Norway'

In [11]:
dados_confiaveis_filtro1

,C1,C2,Time,C3,C4,C5,C6,C7,C8,Confirmed,Country/Region,Date,Deaths,estagio,peak,postinflex,preinflex
0,0.0,0.0,1,0.0,1.0,0.0,0.0,0.0,0.0,3.0,AC,2020-03-19,0.0,5.0,93.0,136.0,50.0
1,0.0,1.0,2,1.0,1.0,1.0,0.0,1.0,1.0,4.0,AC,2020-03-20,0.0,5.0,93.0,136.0,50.0
2,0.0,1.0,3,1.0,1.0,1.0,0.0,1.0,1.0,2.0,AC,2020-03-21,0.0,5.0,93.0,136.0,50.0
3,0.0,1.0,4,1.0,1.0,1.0,0.0,1.0,1.0,2.0,AC,2020-03-22,0.0,5.0,93.0,136.0,50.0
4,0.0,1.0,5,1.0,1.0,1.0,0.0,1.0,1.0,0.0,AC,2020-03-23,0.0,5.0,93.0,136.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18707,1.0,1.0,108,0.0,0.0,0.0,0.0,0.0,1.0,7.0,Yemen,2020-07-26,5.0,1.0,149.0,173.0,124.0
18708,1.0,1.0,109,0.0,0.0,0.0,0.0,0.0,1.0,10.0,Yemen,2020-07-27,4.0,1.0,149.0,173.0,124.0
18709,1.0,1.0,110,0.0,0.0,0.0,0.0,0.0,1.0,12.0,Yemen,2020-07-28,1.0,1.0,149.0,173.0,124.0
18710,1.0,1.0,111,0.0,0.0,0.0,0.0,0.0,1.0,8.0,Yemen,2020-07-29,1.0,1.0,149.0,173.0,124.0


## MULTIVARIEDADE

#### LEITURA DO BANCO DE DADOS SINTÉTICO

In [12]:
nome_arquivo_analise = "synthetic_datasets/Dados_Sinteticos_Casos_Confirmados.csv"
arquivo = os.path.abspath(nome_arquivo_analise)
dados_sinteticos = pd.read_csv(arquivo,  sep="," , decimal=".")
dados_sinteticos

,Data,ID,Time,Regiao de REF,Confirmed
0,2020-03-19,1,1,AC,1.0
1,2020-03-20,1,2,AC,1.0
2,2020-03-21,1,3,AC,0.0
3,2020-03-22,1,4,AC,1.0
4,2020-03-23,1,5,AC,0.0
...,...,...,...,...,...
12001269,2020-10-19,52500,193,Yemen,0.0
12001270,2020-10-20,52500,194,Yemen,0.0
12001271,2020-10-21,52500,195,Yemen,0.0
12001272,2020-10-22,52500,196,Yemen,0.0


In [13]:
numero_de_estimadores = 70000
rate=0.4

#### TREINAMENTO PARA INSERIR DADOS DE ÓBITO

In [14]:
#### DIVISÃO PREDITORES E TARGET
X = dados_confiaveis_filtro1.loc[:, ["Time","Confirmed"]].values
y = dados_confiaveis_filtro1.loc[:, "Deaths"].values

print(X.shape)
print(y.shape)

#### GRADIENT BOOSTING REGRESSOR

regressor_obito = GradientBoostingRegressor(n_estimators= numero_de_estimadores, learning_rate=rate, verbose=1)
regressor_obito.fit(X,y)
score = regressor_obito.score(X,y)
score

(18284, 2)
(18284,)
      Iter       Train Loss   Remaining Time 
         1       13366.3138            7.40m
         2        8804.2302            7.51m
         3        7026.5093            7.47m
         4        6212.8753            7.42m
         5        5791.4204            7.40m
         6        5557.5641            7.38m
         7        5398.9710            7.43m
         8        5292.1057            7.44m
         9        5226.4221            7.43m
        10        5190.9722            7.42m
        20        4206.0271            7.38m
        30        3956.9290            7.37m
        40        3597.4987            7.38m
        50        3347.1553            7.40m
        60        2989.5411            7.41m
        70        2805.2017            7.39m
        80        2663.3698            7.40m
        90        2485.5871            7.40m
       100        2402.1004            7.40m
       200        1646.7327            7.38m
       300        1298.8265       

0.9983338268780608

#### TREINAMENTO PARA INSERIR DADOS DE CONTIGÊNCIA C1

In [15]:
#### DIVISÃO PREDITORES E TARGET
X = dados_confiaveis_filtro1.loc[:, ["Time","Confirmed"]].values
y = dados_confiaveis_filtro1.loc[:, "C1"].values

print(X.shape)
print(y.shape)

#### GRADIENT BOOSTING REGRESSOR

regressor_c1 = GradientBoostingRegressor(n_estimators= numero_de_estimadores, learning_rate=rate, verbose=1)
regressor_c1.fit(X,y)
score = regressor_c1.score(X,y)
score

(18284, 2)
(18284,)
      Iter       Train Loss   Remaining Time 
         1           0.1357            8.53m
         2           0.1280            8.13m
         3           0.1241            7.83m
         4           0.1223            7.67m
         5           0.1215            7.60m
         6           0.1209            7.74m
         7           0.1204            7.72m
         8           0.1200            7.67m
         9           0.1194            7.68m
        10           0.1193            7.67m
        20           0.1175            7.53m
        30           0.1162            7.49m
        40           0.1155            7.49m
        50           0.1149            7.54m
        60           0.1140            7.56m
        70           0.1132            7.53m
        80           0.1126            7.50m
        90           0.1121            7.50m
       100           0.1117            7.49m
       200           0.1072            7.43m
       300           0.1040       

0.69211696958018

#### TREINAMENTO PARA INSERIR DADOS DE CONTIGÊNCIA C2

In [16]:
#### DIVISÃO PREDITORES E TARGET
X = dados_confiaveis_filtro1.loc[:, ["Time","Confirmed"]].values
y = dados_confiaveis_filtro1.loc[:, "C2"].values

print(X.shape)
print(y.shape)

#### GRADIENT BOOSTING REGRESSOR

regressor_c2 = GradientBoostingRegressor(n_estimators= numero_de_estimadores, learning_rate=rate, verbose=1)
regressor_c2.fit(X,y)
score = regressor_c2.score(X,y)
score

(18284, 2)
(18284,)
      Iter       Train Loss   Remaining Time 
         1           0.1601            7.19m
         2           0.1510            7.42m
         3           0.1471            7.43m
         4           0.1447            7.49m
         5           0.1436            7.48m
         6           0.1429            7.50m
         7           0.1426            7.53m
         8           0.1421            7.48m
         9           0.1419            7.53m
        10           0.1415            7.54m
        20           0.1395            7.40m
        30           0.1382            7.37m
        40           0.1371            7.37m
        50           0.1361            7.38m
        60           0.1354            7.39m
        70           0.1347            7.37m
        80           0.1340            7.36m
        90           0.1334            7.37m
       100           0.1329            7.39m
       200           0.1280            7.35m
       300           0.1243       

0.7254010011823515

#### TREINAMENTO PARA INSERIR DADOS DE CONTIGÊNCIA C3

In [17]:
#### DIVISÃO PREDITORES E TARGET
X = dados_confiaveis_filtro1.loc[:, ["Time","Confirmed"]].values
y = dados_confiaveis_filtro1.loc[:, "C3"].values

print(X.shape)
print(y.shape)

#### GRADIENT BOOSTING REGRESSOR

regressor_c3 = GradientBoostingRegressor(n_estimators= numero_de_estimadores ,learning_rate=rate, verbose=1)
regressor_c3.fit(X,y)
score = regressor_c3.score(X,y)
score

(18284, 2)
(18284,)
      Iter       Train Loss   Remaining Time 
         1           0.1277            7.57m
         2           0.1214            7.72m
         3           0.1180            7.63m
         4           0.1166            7.64m
         5           0.1158            7.65m
         6           0.1151            7.66m
         7           0.1147            7.60m
         8           0.1142            7.55m
         9           0.1138            7.63m
        10           0.1136            7.62m
        20           0.1118            7.46m
        30           0.1110            7.47m
        40           0.1101            7.45m
        50           0.1094            7.47m
        60           0.1086            7.46m
        70           0.1079            7.43m
        80           0.1075            7.41m
        90           0.1067            7.43m
       100           0.1061            7.42m
       200           0.1015            7.37m
       300           0.0980       

0.6904837714657697

#### TREINAMENTO PARA INSERIR DADOS DE CONTIGÊNCIA C4

In [18]:
#### DIVISÃO PREDITORES E TARGET
X = dados_confiaveis_filtro1.loc[:, ["Time","Confirmed"]].values
y = dados_confiaveis_filtro1.loc[:, "C4"].values

print(X.shape)
print(y.shape)

#### GRADIENT BOOSTING REGRESSOR

regressor_c4 = GradientBoostingRegressor(n_estimators= numero_de_estimadores ,learning_rate=rate, verbose=1)
regressor_c4.fit(X,y)
score = regressor_c4.score(X,y)
score

(18284, 2)
(18284,)
      Iter       Train Loss   Remaining Time 
         1           0.1635            8.19m
         2           0.1568            7.85m
         3           0.1537            7.96m
         4           0.1516            7.78m
         5           0.1503            7.72m
         6           0.1498            7.67m
         7           0.1493            7.65m
         8           0.1485            7.60m
         9           0.1483            7.56m
        10           0.1479            7.57m
        20           0.1455            7.46m
        30           0.1439            7.48m
        40           0.1424            7.48m
        50           0.1413            7.44m
        60           0.1405            7.46m
        70           0.1395            7.43m
        80           0.1388            7.42m
        90           0.1380            7.41m
       100           0.1374            7.42m
       200           0.1322            7.37m
       300           0.1280       

0.7081572726592411

#### TREINAMENTO PARA INSERIR DADOS DE CONTIGÊNCIA C5

In [19]:
#### DIVISÃO PREDITORES E TARGET
X = dados_confiaveis_filtro1.loc[:, ["Time","Confirmed"]].values
y = dados_confiaveis_filtro1.loc[:, "C5"].values

print(X.shape)
print(y.shape)

#### GRADIENT BOOSTING REGRESSOR

regressor_c5 = GradientBoostingRegressor(n_estimators = numero_de_estimadores, learning_rate=rate, verbose=1)
regressor_c5.fit(X,y)
score = regressor_c5.score(X,y)
score

(18284, 2)
(18284,)
      Iter       Train Loss   Remaining Time 
         1           0.2308            7.40m
         2           0.2218            7.48m
         3           0.2170            7.45m
         4           0.2148            7.71m
         5           0.2132            7.60m
         6           0.2120            7.64m
         7           0.2113            7.58m
         8           0.2110            7.60m
         9           0.2107            7.62m
        10           0.2106            7.63m
        20           0.2078            7.48m
        30           0.2060            7.45m
        40           0.2046            7.42m
        50           0.2030            7.40m
        60           0.2017            7.39m
        70           0.2004            7.37m
        80           0.1996            7.39m
        90           0.1984            7.39m
       100           0.1973            7.39m
       200           0.1897            7.37m
       300           0.1837       

0.7592558146983287

#### TREINAMENTO PARA INSERIR DADOS DE CONTIGÊNCIA C6

In [20]:
#### DIVISÃO PREDITORES E TARGET
X = dados_confiaveis_filtro1.loc[:, ["Time","Confirmed"]].values
y = dados_confiaveis_filtro1.loc[:, "C6"].values

print(X.shape)
print(y.shape)

#### GRADIENT BOOSTING REGRESSOR

regressor_c6 = GradientBoostingRegressor(n_estimators = numero_de_estimadores, learning_rate=rate, verbose=1)
regressor_c6.fit(X,y)
score = regressor_c6.score(X,y)
score

(18284, 2)
(18284,)
      Iter       Train Loss   Remaining Time 
         1           0.1984            7.15m
         2           0.1859            7.31m
         3           0.1798            7.35m
         4           0.1776            7.37m
         5           0.1756            7.43m
         6           0.1749            7.42m
         7           0.1746            7.60m
         8           0.1739            7.58m
         9           0.1735            7.57m
        10           0.1732            7.53m
        20           0.1710            7.45m
        30           0.1691            7.44m
        40           0.1678            7.50m
        50           0.1669            7.52m
        60           0.1659            7.50m
        70           0.1649            7.47m
        80           0.1642            7.46m
        90           0.1635            7.46m
       100           0.1625            7.44m
       200           0.1558            7.38m
       300           0.1509       

0.7585413130627375

#### TREINAMENTO PARA INSERIR DADOS DE CONTIGÊNCIA C7

In [21]:
#### DIVISÃO PREDITORES E TARGET
X = dados_confiaveis_filtro1.loc[:, ["Time","Confirmed"]].values
y = dados_confiaveis_filtro1.loc[:, "C7"].values

print(X.shape)
print(y.shape)

#### GRADIENT BOOSTING REGRESSOR

regressor_c7 = GradientBoostingRegressor(n_estimators = numero_de_estimadores, learning_rate=rate, verbose=1)
regressor_c7.fit(X,y)
score = regressor_c7.score(X,y)
score

(18284, 2)
(18284,)
      Iter       Train Loss   Remaining Time 
         1           0.1991            8.76m
         2           0.1866            8.14m
         3           0.1801            7.83m
         4           0.1771            7.76m
         5           0.1754            7.81m
         6           0.1747            7.95m
         7           0.1736            7.90m
         8           0.1731            7.94m
         9           0.1727            7.87m
        10           0.1725            7.95m
        20           0.1701            7.76m
        30           0.1687            7.68m
        40           0.1673            7.67m
        50           0.1661            7.59m
        60           0.1654            7.56m
        70           0.1646            7.54m
        80           0.1639            7.51m
        90           0.1631            7.51m
       100           0.1623            7.49m
       200           0.1555            7.42m
       300           0.1513       

0.748393154574302

#### TREINAMENTO PARA INSERIR DADOS DE CONTIGÊNCIA C8

In [22]:
#### DIVISÃO PREDITORES E TARGET
X = dados_confiaveis_filtro1.loc[:, ["Time","Confirmed"]].values
y = dados_confiaveis_filtro1.loc[:, "C8"].values

print(X.shape)
print(y.shape)

#### GRADIENT BOOSTING REGRESSOR

regressor_c8 = GradientBoostingRegressor(n_estimators= numero_de_estimadores, learning_rate=rate, verbose=1)
regressor_c8.fit(X,y)
score = regressor_c8.score(X,y)
score

(18284, 2)
(18284,)
      Iter       Train Loss   Remaining Time 
         1           0.1297            7.43m
         2           0.1271            8.08m
         3           0.1257            7.80m
         4           0.1249            7.94m
         5           0.1243            7.78m
         6           0.1241            7.78m
         7           0.1238            7.68m
         8           0.1235            7.67m
         9           0.1232            7.74m
        10           0.1230            7.73m
        20           0.1210            7.53m
        30           0.1199            7.43m
        40           0.1185            7.42m
        50           0.1175            7.41m
        60           0.1164            7.43m
        70           0.1156            7.43m
        80           0.1150            7.41m
        90           0.1144            7.41m
       100           0.1137            7.42m
       200           0.1082            7.36m
       300           0.1042       

0.6998100871563981

## ADIÇÃO DA MULTIVARIEDADE

In [23]:
start_time = datetime.now() 

In [24]:
#PREDIÇÃO (ADICIONANDO OS CASOS DE MORTE AOS DADOS SINTÉTICOS)
morte_estimadas = regressor_obito.predict(dados_sinteticos.loc[:,["Time","Confirmed"]].values)
morte_estimadas = np.around(morte_estimadas,0)
dados_sinteticos["Deaths"] = morte_estimadas
dados_sinteticos
print("CASOS DE MORTE - DONE")

CASOS DE MORTE - DONE


In [25]:
time_elapsed = datetime.now() - start_time 
print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))

Time elapsed (hh:mm:ss.ms) 1:13:10.136077


In [26]:
start_time = datetime.now() 

In [27]:
#PREDIÇÃO (ADICIONANDO DADOS DE C1 SITUAÇÃO HIPOTÉTICA)
c1_estimadas = regressor_c1.predict(dados_sinteticos.loc[:,["Time","Confirmed"]].values)
c1_estimadas = np.around(c1_estimadas,0)
dados_sinteticos["C1"] = c1_estimadas
print("C1 - DONE")

C1 - DONE


In [28]:
time_elapsed = datetime.now() - start_time 
print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))

Time elapsed (hh:mm:ss.ms) 1:15:36.607979


In [29]:
start_time = datetime.now() 

In [30]:
#PREDIÇÃO (ADICIONANDO DADOS DE C2 SITUAÇÃO HIPOTÉTICA)
c2_estimadas = regressor_c2.predict(dados_sinteticos.loc[:,["Time","Confirmed"]].values)
c2_estimadas = np.around(c2_estimadas,0)
dados_sinteticos["C2"] = c2_estimadas
print("C2 - DONE")

C2 - DONE


In [31]:
time_elapsed = datetime.now() - start_time 
print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))

Time elapsed (hh:mm:ss.ms) 1:15:18.309140


In [32]:
start_time = datetime.now() 

In [33]:
#PREDIÇÃO (ADICIONANDO DADOS DE C3 SITUAÇÃO HIPOTÉTICA)
c3_estimadas = regressor_c3.predict(dados_sinteticos.loc[:,["Time","Confirmed"]].values)
c3_estimadas = np.around(c3_estimadas,0)
dados_sinteticos["C3"] = c3_estimadas
print("C3 - DONE")

C3 - DONE


In [34]:
time_elapsed = datetime.now() - start_time 
print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))

Time elapsed (hh:mm:ss.ms) 1:16:00.026132


In [35]:
start_time = datetime.now() 

In [36]:
#PREDIÇÃO (ADICIONANDO DADOS DE C4 SITUAÇÃO HIPOTÉTICA)
c4_estimadas = regressor_c4.predict(dados_sinteticos.loc[:,["Time","Confirmed"]].values)
c4_estimadas = np.around(c4_estimadas,0)
dados_sinteticos["C4"] = c4_estimadas
print("C4 - DONE")

C4 - DONE


In [37]:
time_elapsed = datetime.now() - start_time 
print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))

Time elapsed (hh:mm:ss.ms) 1:15:48.036318


In [38]:
start_time = datetime.now() 

In [39]:
#PREDIÇÃO (ADICIONANDO DADOS DE C5 SITUAÇÃO HIPOTÉTICA)
c5_estimadas = regressor_c5.predict(dados_sinteticos.loc[:,["Time","Confirmed"]].values)
c5_estimadas = np.around(c5_estimadas,0)
dados_sinteticos["C5"] = c5_estimadas
print("C5 - DONE")

C5 - DONE


In [40]:
time_elapsed = datetime.now() - start_time 
print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))

Time elapsed (hh:mm:ss.ms) 1:15:45.766954


In [41]:
start_time = datetime.now() 

In [42]:
#PREDIÇÃO (ADICIONANDO DADOS DE C6 SITUAÇÃO HIPOTÉTICA)
c6_estimadas = regressor_c6.predict(dados_sinteticos.loc[:,["Time","Confirmed"]].values)
c6_estimadas = np.around(c6_estimadas,0)
dados_sinteticos["C6"] = c6_estimadas
print("C6 - DONE")

C6 - DONE


In [43]:
time_elapsed = datetime.now() - start_time 
print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))

Time elapsed (hh:mm:ss.ms) 1:15:47.833070


In [44]:
start_time = datetime.now() 

In [45]:
#PREDIÇÃO (ADICIONANDO DADOS DE C7 SITUAÇÃO HIPOTÉTICA)
c7_estimadas = regressor_c7.predict(dados_sinteticos.loc[:,["Time","Confirmed"]].values)
c7_estimadas = np.around(c7_estimadas,0)
dados_sinteticos["C7"] = c7_estimadas
print("C7 - DONE")

C7 - DONE


In [46]:
time_elapsed = datetime.now() - start_time 
print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))

Time elapsed (hh:mm:ss.ms) 1:15:57.104162


In [47]:
start_time = datetime.now() 

In [48]:
#PREDIÇÃO (ADICIONANDO DADOS DE C8 SITUAÇÃO HIPOTÉTICA)
c8_estimadas = regressor_c8.predict(dados_sinteticos.loc[:,["Time","Confirmed"]].values)
c8_estimadas = np.around(c8_estimadas,0)
dados_sinteticos["C8"] = c8_estimadas
print("C8 - DONE")

C8 - DONE


In [49]:
time_elapsed = datetime.now() - start_time 
print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))

Time elapsed (hh:mm:ss.ms) 1:15:24.183985


In [50]:
print("TREINAMENTO CONCLUÍDO")

TREINAMENTO CONCLUÍDO


In [51]:
dados_sinteticos.head()

,Data,ID,Time,Regiao de REF,Confirmed,Deaths,C1,C2,C3,C4,C5,C6,C7,C8
0,2020-03-19,1,1,AC,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-03-20,1,2,AC,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-03-21,1,3,AC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-03-22,1,4,AC,1.0,0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0
4,2020-03-23,1,5,AC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### VERIFICAÇÃO BÁSICA 

In [52]:
dados_sinteticos.isnull().sum()

Data             0
ID               0
Time             0
Regiao de REF    0
Confirmed        0
Deaths           0
C1               0
C2               0
C3               0
C4               0
C5               0
C6               0
C7               0
C8               0
dtype: int64

In [53]:
dados_sinteticos.min()

Data             2020-01-23
ID                        1
Time                      1
Regiao de REF            AC
Confirmed                 0
Deaths                 -471
C1                       -3
C2                       -3
C3                       -1
C4                       -1
C5                       -4
C6                       -4
C7                       -3
C8                       -2
dtype: object

#### CORREÇÕES BÁSICAS

In [54]:
dados_sinteticos_corrigido = dados_sinteticos.copy()

In [55]:
cols = ["Deaths","C1","C2","C3","C4","C5","C6","C7","C8"]
dados_sinteticos_corrigido[cols] = dados_sinteticos_corrigido[cols].clip(lower=0)

In [56]:
dados_sinteticos_corrigido.min()

Data             2020-01-23
ID                        1
Time                      1
Regiao de REF            AC
Confirmed                 0
Deaths                    0
C1                        0
C2                        0
C3                        0
C4                        0
C5                        0
C6                        0
C7                        0
C8                        0
dtype: object

In [57]:
dados_sinteticos_corrigido[cols] = dados_sinteticos_corrigido[cols].round(0)

In [58]:
dados_sinteticos_corrigido.head()

,Data,ID,Time,Regiao de REF,Confirmed,Deaths,C1,C2,C3,C4,C5,C6,C7,C8
0,2020-03-19,1,1,AC,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-03-20,1,2,AC,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-03-21,1,3,AC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-03-22,1,4,AC,1.0,0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0
4,2020-03-23,1,5,AC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### SALVANDO O BANCO DE DADOS

In [59]:
dados_sinteticos_corrigido.to_csv("synthetic_datasets/Dados_Sinteticos_Multivariado.csv",index=False)